# Exploring data with BigQuery
## Setup Working Directory

In [7]:
import sys
import os
import pathlib

workingdir=os.getcwd()
print(workingdir)
d=[d for d in os.listdir(workingdir)]
n=0
while not set(['notebook']).issubset(set(d)):
    workingdir=str(pathlib.Path(workingdir).parents[0])
    print(workingdir)
    
    
    
    
    d=[d for d in os.listdir(str(workingdir))]
    n+=1
    if n>5:
        break
sys.path.insert(0, workingdir)
os.chdir(workingdir)

C:\Users\C172685\Projects\Code\proj_NLP_text_classification_with_GCP


## Import libraries

In [8]:
# Import the client library
import google.cloud.dlp
import os
import subprocess
import json 
import requests
import getpass
import urllib.parse

## Defined GCP env variables

In [9]:
os.environ['PROJECT_ID'] = subprocess.run('gcloud config list project --format "value(core.project)"', shell=True, check=True, stdout=subprocess.PIPE).stdout.decode().replace('\n', '').replace('\r', '')
print(os.environ['PROJECT_ID'])

axa-ch-machine-learning-dev


In [10]:
# bucket storage name
os.environ['BUCKET_NAME']='axa-ch-machine-learning-poc-dev'

In [11]:
try:
    tmp=os.environ['PROJECT_ID']
except:
    print('Env variable PROJECT not defined!') 

try:
    tmp=os.environ['BUCKET_NAME']
except:
    print('Env variable BUCKET_NAME not defined!') 
    
try:    
    tmp=os.environ['GOOGLE_APPLICATION_CREDENTIALS']
except:
    print('Env variable GOOGLE_APPLICATION_CREDENTIALS not defined!') 

try:
    tmp=os.environ['REQUESTS_CA_BUNDLE']
except:
    print('Env variable REQUESTS_CA_BUNDLE not defined!') 

try:
    tmp=os.environ['AXA_CH_CA_BUNDLE']
except:
    print('Env variable AXA_CA_CA_BUNDLE not defined!') 

In [12]:
print(os.environ['PROJECT_ID'])
print(os.environ['BUCKET_NAME'])
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
print(os.environ['REQUESTS_CA_BUNDLE'])
print(os.environ['AXA_CH_CA_BUNDLE'])

axa-ch-machine-learning-dev
axa-ch-machine-learning-poc-dev
C:/eplatform/tools/GCP/credentials/machine-learning-poc-dev-5fdc233c6e8b.json
C:/ProgramData/Anaconda3/Lib/site-packages/certifi/cacert.pem
C:\eplatform\tools\google-cloud-sdk\axa_ch\ca_certs_axa_ch


## Setup Proxy (If needed)

## Merge CA and AXA CH CA certificates (If needed)

In [13]:
missing_AXA_CA=False
with open(os.environ['REQUESTS_CA_BUNDLE'])as ca_file:
    for line in ca_file:
        if 'CN=AXA-CH-Proxy-Issuing' in line:
            print(line)
            missing_AXA_CA=True

# Issuer: CN=AXA-CH-Proxy-Issuing-CA11 O=AXA Services



In [14]:
#update the CA file if AXA CA is missing
if not missing_AXA_CA:
    with open(os.environ['REQUESTS_CA_BUNDLE'], 'a+')as ca_file:
        with open(os.environ['AXA_CH_CA_BUNDLE'])as axa_ca_file:
            for line in axa_ca_file:
                ca_file.write(line)
        

In [15]:
# check
with open(os.environ['REQUESTS_CA_BUNDLE'])as ca_file:
    for line in ca_file:
        if 'CN=AXA-CH-Proxy-Issuing' in line:
            print(line)

# Issuer: CN=AXA-CH-Proxy-Issuing-CA11 O=AXA Services



## Testing everything

In [16]:
# use env variables
use_proxy='Y'
proxies = {
    'https': os.environ['HTTPS_PROXY'],    
    'http': os.environ['HTTP_PROXY']
}  

In [17]:
list_url=['https://www.google.com',
          'http://www.google.com',
          'https://www.example.com',
          'http://www.example.com',
          'https://github.com/j0hannes/cutter-ng']

for url in list_url:
    print('')
    print('trying to access:'+url)
    try:
        if use_proxy=='N':
            r = requests.get(url)
        else:
            # SSL deactivated
            #r = requests.get(url,proxies=proxies,verify=False)
            r = requests.get(url,proxies=proxies,verify=True)
            
        if r.status_code == requests.codes.ok:
            print('=>OK',r.headers['content-type'])
        else:
            # 407 Proxy Authentication Required
            print ('=> ??', r.status_code)
    except Exception as inst:
        print('=>FAILED')
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args
        print(inst)


trying to access:https://www.google.com
=>OK text/html; charset=ISO-8859-1

trying to access:http://www.google.com
=>OK text/html; charset=ISO-8859-1

trying to access:https://www.example.com
=>OK text/html; charset=UTF-8

trying to access:http://www.example.com
=>OK text/html; charset=UTF-8

trying to access:https://github.com/j0hannes/cutter-ng
=>OK text/html; charset=utf-8


## Test of the proxy with Gloud Storage

In [19]:
from google.cloud import storage
storage_client = storage.Client()
buckets = storage_client.list_buckets()

for bucket in buckets:
    print(bucket.name)
    

blobs = storage_client.list_blobs(os.environ['BUCKET_NAME'])
for blob in blobs:
    print(blob.name)
    break

artifacts.axa-ch-machine-learning-dev.appspot.com
axa-ch-machine-learning-dev-lcm
axa-ch-machine-learning-dev-vcm
axa-ch-machine-learning-dev.appspot.com
axa-ch-machine-learning-dev_cloudbuild
axa-ch-machine-learning-poc-dev
axa-ch-project-build-cache
cloud-build-slack
dataprep-staging-216e5014-18fe-4dea-a142-09b6ebf6d667
europe-west1-test-b090db08-bucket
staging.axa-ch-machine-learning-dev.appspot.com
certificates/


## Test of the proxy with BigQuery

In [20]:
from google.cloud import bigquery
client = bigquery.Client()

query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100 * SUM(IF(answer_count > 0, 1, 0)) / COUNT(*), 1) AS Percent_Questions_with_Answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2016
ORDER BY
  Year
"""
#df = client.query(query).to_dataframe()
#df.head()
query_job = client.query(query)
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((2009, 342995, 99.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2010, 693152, 99.1), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2011, 1198188, 97.2), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2012, 1642244, 94.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2013, 2055803, 91.7), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2014, 2159044, 88.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2015, 2213042, 86.4), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
